In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import recordlinkage

REQD_COLUMNS = ['name', 'street', 'city', 'state', 'zip']

In [10]:
city_df = pd.read_csv("./data/UE-cities.csv")
df_sba = pd.read_csv("./data/PPP Data 150k plus.csv")

In [11]:
df_sba['Zip'] = df_sba['Zip'].astype(int).astype(str).str.zfill(5)

df_a = df_sba[['BusinessName', 'Address', 'City', 'State', 'Zip']]
df_a.columns = REQD_COLUMNS

df_a = df_a.fillna({'name':""})

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [4]:
for url in city_df['url'][0:1]:
    cityname = url.split("/")[-1]
    cityfile = "../Extreme-Uber-Eats-Scraping/scraped-data/restaurant-details/" + cityname + "-details.csv"
    df_ue = pd.read_csv(cityfile, dtype=str)

In [5]:
df_a.head(1)
df_ue.head(1)

,name,street,city,state,zip
0,1186 BROADWAY RESTAURANT LLC,30 W 26th Street Floor 12,NEW YORK,NY,10010


,name,url,city,cuisine,avgRating,numReviews,priceRange,latitude,longitude,telephone,postalCode,streetAddress,addressLocality,addressRegion,openingHoursSpecification,addressString,geoString,ratingString
0,"Mr. Gatti's Pizza - Abilene, TX",https://www.ubereats.com/abilene/food-delivery...,abilene,"['Pizza', 'Comfort Food', 'Italian']",NaN,NaN,$,32.4175466,-99.74861399999999,+13256926326,79605,2665 Buffalo Gap Rd,Abilene,TX,"[{'@type': 'OpeningHoursSpecification', 'dayOf...","{'@type': 'PostalAddress', 'addressLocality': ...","{'@type': 'GeoCoordinates', 'latitude': 32.417...",{}


In [7]:
df_ue['name'] = df_ue['name'].str.upper()
df_ue['addressLocality'] = df_ue['addressLocality'].str.upper()
df_ue['addressRegion'] = df_ue['addressRegion'].str.upper()

df_b = df_ue[['name', 'streetAddress', 'addressLocality', 'addressRegion', 'postalCode']]
df_b.columns = REQD_COLUMNS

df_b = df_b.dropna(subset=['name'])
df_b = df_b.fillna({'street':"", 'city':"", 'state':"", 'zip':""})

df_b['city'] = df_b['city'].value_counts().keys()[0]
df_b['state'] = df_b['state'].value_counts().keys()[0]
df_b['zip'] = df_b['zip'].apply(lambda x: "" if len(x) != 5 else x)

In [8]:
df_a.head(1)
df_b.head(1)`

,name,street,city,state,zip
0,1186 BROADWAY RESTAURANT LLC,30 W 26th Street Floor 12,NEW YORK,NY,10010


,name,street,city,state,zip
0,"MR. GATTI'S PIZZA - ABILENE, TX",2665 Buffalo Gap Rd,ABILENE,TX,79605


In [9]:
indexer = recordlinkage.Index()
indexer.block('zip')

candidates = indexer.index(df_a, df_b)
print(len(candidates))

compare = recordlinkage.Compare()
# compare.exact('city', 'Provider City', label='City')
compare.string('name',
            'name',
            method='jarowinkler',
            threshold=0.75,
            label='name_low')
compare.string('name',
            'name',
            method='jarowinkler',
            threshold=0.9,
            label='name_high')
compare.string('street',
            'street',
            method='jarowinkler',
            threshold=0.85,
            label='street')
# compare.exact('zip',
#             'zip',
#             label='zip')
features = compare.compute(candidates, df_a, df_b)
features.sum(axis=1).value_counts().sort_index(ascending=False)

potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['score'] = potential_matches.loc[:, 'name_low':'street'].sum(axis=1)
potential_matches.head(5)

<Index>

0


<Compare>

<Compare>

<Compare>

Series([], dtype: int64)

,level_0,level_1,name_low,name_high,street,score
